# bc_parser.py
This notebook parses barcodes from Illumina sequencing data. Input is listed in [data/samplelist.csv](./data/samplelist.csv). Output is stored in the [results/](./results/) directory. For more details, see [README](./README.md).

Imports

In [1]:
import pandas as pd

Visualization styles

In [2]:
# Dutch-field from https://gist.github.com/afcotroneo/ca9716f755128b5e9b2ed1fe4186f4df
palette = ["#e60049", "#0bb4ff", "#50e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0"]

## input data
### Samples

In [3]:
samplelist = './data/samplelist.csv'
samples_df = pd.read_csv(samplelist, comment='#')
display(samples_df)

,sample,fastq_file,upstream_seq,downstream_seq
0,BC20_Perth09_bcHA_RepA_01,/Users/dbacsik/Downloads/SWGG9B_info/SWGG9B_ra...,TGCAACATCTGTATATAATGAG,tggatttcctttgccatatc
1,BC20_Perth09_bcHA_RepB_02,/Users/dbacsik/Downloads/SWGG9B_info/SWGG9B_ra...,TGCAACATCTGTATATAATGAG,tggatttcctttgccatatc
